# SQL Basic Reporting

Welcome to **Basic Reporting in SQL**!

In this notebook, you’ll learn how to build **practical SQL reports**, the kind of queries that combine multiple concepts to return exactly the information you need.

We’ll be writing longer, more structured queries that:
- pull data from several tables
- group and aggregate results
- filter and sort summaries

In this part, we’ll:
- get familiar with the database we’ll be working with
- review a few key concepts
- then move on to building real SQL reports

Let’s get started.


## **SQL Environment Setup (do not edit)**

In [1]:
# @title

%%capture
!mkdir -p notebook_lib
!wget -q -O notebook_lib/sql_runner.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/8021f5c05b7d973b8db549a1398a3c9a5c7829d5/notebook_lib/sql_runner.py
!wget -q -O notebook_lib/validators.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/23a70af86315e13d2a8fca953970b60c47de690f/notebook_lib/validators.py

from notebook_lib.sql_runner import make_sql_runner
from notebook_lib.validators import make_df_validator_nospoilers, check_process_rules

import sqlite3
import pandas as pd
from pathlib import Path


In [2]:
# @title

DB_FILE = 'class.db'

if DB_FILE != ":memory:":
    Path(DB_FILE).unlink(missing_ok=True)

conn = sqlite3.connect(DB_FILE)
conn.execute("PRAGMA foreign_keys = ON;")

conn.executescript(r"""
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS suppliers;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS categories;
DROP TABLE IF EXISTS employees;

-- 1) employees
CREATE TABLE employees (
  employee_id   INTEGER PRIMARY KEY,
  last_name     TEXT NOT NULL,
  first_name    TEXT NOT NULL,
  title         TEXT,
  birth_date    TEXT,              -- YYYY-MM-DD
  hire_date     TEXT,              -- YYYY-MM-DD
  address       TEXT,
  city          TEXT,
  region        TEXT,
  postal_code   TEXT,
  country       TEXT,
  reports_to    INTEGER,
  FOREIGN KEY (reports_to) REFERENCES employees(employee_id)
    ON UPDATE CASCADE
    ON DELETE SET NULL
);

-- 2) categories
CREATE TABLE categories (
  category_id    INTEGER PRIMARY KEY,
  category_name  TEXT NOT NULL,
  description    TEXT
);

-- 3) customers
CREATE TABLE customers (
  customer_id    TEXT PRIMARY KEY,   -- e.g., ALFKI
  company_name   TEXT NOT NULL,
  contact_name   TEXT,
  contact_title  TEXT,
  address        TEXT,
  city           TEXT,
  region         TEXT,
  postal_code    TEXT,
  country        TEXT,
  fax            TEXT
);

-- 4) suppliers
CREATE TABLE suppliers (
  supplier_id   INTEGER PRIMARY KEY,
  company_name  TEXT NOT NULL,
  address       TEXT,
  city          TEXT,
  region        TEXT,
  postal_code   TEXT,
  country       TEXT
);

-- 5) products
CREATE TABLE products (
  product_id         INTEGER PRIMARY KEY,
  product_name       TEXT NOT NULL,
  supplier_id        INTEGER NOT NULL,
  category_id        INTEGER NOT NULL,
  quantity_per_unit  TEXT,
  unit_price         REAL NOT NULL CHECK (unit_price >= 0),
  units_in_stock     INTEGER NOT NULL DEFAULT 0 CHECK (units_in_stock >= 0),
  units_on_order     INTEGER NOT NULL DEFAULT 0 CHECK (units_on_order >= 0),
  discontinued       TEXT NOT NULL CHECK (discontinued IN ('t','f')),
  FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT,
  FOREIGN KEY (category_id) REFERENCES categories(category_id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
);

-- 6) orders
CREATE TABLE orders (
  order_id           INTEGER PRIMARY KEY,
  customer_id        TEXT NOT NULL,
  employee_id        INTEGER NOT NULL,
  order_date         TEXT,          -- YYYY-MM-DD
  shipped_date       TEXT,          -- YYYY-MM-DD (nullable)
  ship_via           INTEGER,       -- no shippers table in your 7-table set; keep as plain INTEGER
  freight            REAL CHECK (freight >= 0),
  ship_address       TEXT,
  ship_city          TEXT,
  ship_region        TEXT,
  ship_postal_code   TEXT,
  ship_country       TEXT,
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT,
  FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
);

-- 7) order_items
CREATE TABLE order_items (
  order_id    INTEGER NOT NULL,
  product_id  INTEGER NOT NULL,
  unit_price  REAL NOT NULL CHECK (unit_price >= 0),
  quantity    INTEGER NOT NULL CHECK (quantity > 0),
  discount    REAL NOT NULL DEFAULT 0 CHECK (discount >= 0 AND discount <= 1),
  PRIMARY KEY (order_id, product_id),
  FOREIGN KEY (order_id) REFERENCES orders(order_id)
    ON UPDATE CASCADE
    ON DELETE CASCADE,
  FOREIGN KEY (product_id) REFERENCES products(product_id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
);

-- Helpful indexes (not required, but makes joins faster)
CREATE INDEX idx_orders_customer_id   ON orders(customer_id);
CREATE INDEX idx_orders_employee_id   ON orders(employee_id);
CREATE INDEX idx_products_supplier_id ON products(supplier_id);
CREATE INDEX idx_products_category_id ON products(category_id);
CREATE INDEX idx_order_items_product  ON order_items(product_id);


INSERT INTO employees (
  employee_id, last_name, first_name, title,
  birth_date, hire_date,
  address, city, region, postal_code, country,
  reports_to
) VALUES
(1,'Davolio','Nancy','Sales Representative','1968-12-08','2012-05-01','507 - 20th Ave. E. Apt. 2A','Seattle','WA','98122','USA',2),
(2,'Fuller','Andrew','Vice President, Sales','1972-02-19','2012-08-14','908 W. Capital Way','Tacoma','WA','98401','USA',NULL),
(3,'Smith','John','Sales Representative','1983-08-30','2012-04-01','722 Moss Bay Blvd.','Kirkland','WA','98033','USA',2),
(4,'Peacock','Margaret','Sales Representative','1957-09-19','2013-05-03','4110 Old Redmond Rd.','Redmond','WA','98052','USA',2),
(5,'Buchanan','Steven','Sales Manager','1975-03-04','2013-10-17','14 Garrett Hill','London',NULL,'SW1 8JR','UK',2),
(6,'Suyama','Michael','Sales Representative','1983-07-02','2013-10-17','Coventry House Miner Rd.','London',NULL,'EC2 7JR','UK',5),
(7,'King','Robert','Sales Representative','1980-05-29','2014-01-02','Edgeham Hollow Winchester Way','London',NULL,'RG1 9SP','UK',5),
(8,'Callahan','Laura','Inside Sales Coordinator','1978-01-09','2014-03-05','4726 - 11th Ave. N.E.','Seattle','WA','98105','USA',2),
(9,'Dodsworth','Anne','Sales Representative','1986-01-27','2014-11-15','7 Houndstooth Rd.','London',NULL,'WG2 7LT','UK',5),
(10,'Smith','John','Sales Representative','1994-08-30','2017-03-21','22 Abbey Rd','London',NULL,'NW6 5JG','UK',2);


INSERT INTO customers (
  customer_id, company_name, contact_name, contact_title,
  address, city, region, postal_code, country, fax
) VALUES
('ALFKI','Alfreds Futterkiste','Maria Anders','Sales Representative','Obere Str. 57','Berlin',NULL,'12209','Germany','030-0076545'),
('ANATR','Ana Trujillo Emparedados y helados','Ana Trujillo','Owner','Avda. de la Constitución 2222','México D.F.',NULL,'05021','Mexico','(5) 555-3745'),
('ANTON','Antonio Moreno Taquería','Antonio Moreno','Owner','Mataderos 2312','México D.F.',NULL,'05023','Mexico',NULL),
('AROUT','Around the Horn','Thomas Hardy','Sales Representative','120 Hanover Sq.','London',NULL,'WA1 1DP','UK','(171) 555-6750'),
('BERGS','Berglunds snabbköp','Christina Berglund','Order Administrator','Berguvsvägen 8','Luleå',NULL,'S-958 22','Sweden','0921-12 34 67'),
('BLAUS','Blauer See Delikatessen','Hanna Moos','Sales Representative','Forsterstr. 57','Mannheim',NULL,'68306','Germany','0621-08924'),
('BLONP','Blondesddsl père et fils','Frédérique Citeaux','Marketing Manager','24, place Kléber','Strasbourg',NULL,'67000','France','88.60.15.32'),
('BOLID','Bólido Comidas preparadas','Martín Sommer','Owner','C/ Araquil, 67','Madrid',NULL,'28023','Spain','(91) 555 91 99'),
('BONAP','Bon app''','Laurence Lebihan','Owner','12, rue des Bouchers','Marseille',NULL,'13008','France','91.24.45.41'),
('BOTTM','Bottom-Dollar Markets','Elizabeth Lincoln','Accounting Manager','23 Tsawassen Blvd.','Tsawassen','BC','T2F 8M4','Canada','(604) 555-3745'),
('BSBEV','B''s Beverages','Victoria Ashworth','Sales Representative','Fauntleroy Circus','London',NULL,'EC2 5NT','UK',NULL),
('CACTU','Cactus Comidas para llevar','Patricio Simpson','Sales Agent','Cerrito 333','Buenos Aires',NULL,'1010','Argentina','(1) 135-4892'),
('CENTC','Centro comercial Moctezuma','Francisco Chang','Marketing Manager','Sierras de Granada 9993','México D.F.',NULL,'05022','Mexico','(5) 555-7293'),
('CHOPS','Chop-suey Chinese','Yang Wang','Owner','Hauptstr. 29','Bern',NULL,'3012','Switzerland',NULL),
('COMMI','Comércio Mineiro','Pedro Afonso','Sales Associate','Av. dos Lusíadas, 23','Sao Paulo','SP','05432-043','Brazil',NULL),
('CONSH','Consolidated Holdings','Elizabeth Brown','Sales Representative','Berkeley Gardens 12 Brewery','London',NULL,'WX1 6LT','UK','(171) 555-9199'),
('DRACD','Drachenblut Delikatessen','Sven Ottlieb','Order Administrator','Walserweg 21','Aachen',NULL,'52066','Germany','0241-059428'),
('DUMON','Du monde entier','Janine Labrune','Owner','67, rue des Cinquante Otages','Nantes',NULL,'44000','France','40.67.89.89'),
('EASTC','Eastern Connection','Ann Devon','Sales Agent','35 King George','London',NULL,'WX3 6FW','UK','(171) 555-3373'),
('ERNSH','Ernst Handel','Roland Mendel','Sales Manager','Kirchgasse 6','Graz',NULL,'8010','Austria','7675-3426'),
('FAMIA','Familia Arquibaldo','Aria Cruz','Marketing Assistant','Rua Orós, 92','Sao Paulo','SP','05442-030','Brazil',NULL),
('FISSA','FISSA Fabrica Inter. Salchichas S.A.','Diego Roel','Accounting Manager','C/ Moralzarzal, 86','Madrid',NULL,'28034','Spain','(91) 555 55 93'),
('FOLIG','Folies gourmandes','Martine Rancé','Assistant Sales Agent','184, chaussée de Tournai','Lille',NULL,'59000','France','20.16.10.17'),
('FOLKO','Folk och fä HB','Maria Larsson','Owner','Åkergatan 24','Bräcke',NULL,'S-844 67','Sweden',NULL),
('FRANK','Frankenversand','Peter Franken','Marketing Manager','Berliner Platz 43','München',NULL,'80805','Germany','089-0877451'),
('FRANR','France restauration','Carine Schmitt','Marketing Manager','54, rue Royale','Nantes',NULL,'44000','France','40.32.21.20'),
('FRANS','Franchi S.p.A.','Paolo Accorti','Sales Representative','Via Monte Bianco 34','Torino',NULL,'10100','Italy','011-4988261'),
('FURIB','Furia Bacalhau e Frutos do Mar','Lino Rodriguez','Sales Manager','Jardim das rosas n. 32','Lisboa',NULL,'1675','Portugal','(1) 354-2535'),
('GALED','Galería del gastrónomo','Eduardo Saavedra','Marketing Manager','Rambla de Cataluña, 23','Barcelona',NULL,'08022','Spain','(93) 203 4561'),
('GODOS','Godos Cocina Típica','José Pedro Freyre','Sales Manager','C/ Romero, 33','Sevilla',NULL,'41101','Spain',NULL),
('GOURL','Gourmet Lanchonetes','André Fonseca','Sales Associate','Av. Brasil, 442','Campinas','SP','04876-786','Brazil',NULL),
('GREAL','Great Lakes Food Market','Howard Snyder','Marketing Manager','2732 Baker Blvd.','Eugene','OR','97403','USA',NULL),
('GROSR','GROSELLA-Restaurante','Manuel Pereira','Owner','5ª Ave. Los Palos Grandes','Caracas','DF','1081','Venezuela','(2) 283-3397'),
('HANAR','Hanari Carnes','Mario Pontes','Accounting Manager','Rua do Paço, 67','Rio de Janeiro','RJ','05454-876','Brazil','(21) 555-8765'),
('HILAA','HILARION-Abastos','Carlos Hernández','Sales Representative','Carrera 22 con Ave. Carlos Soublette #8-35','San Cristóbal','Táchira','5022','Venezuela','(5) 555-1948'),
('HUNGC','Hungry Coyote Import Store','Yoshi Latimer','Sales Representative','City Center Plaza 516 Main St.','Elgin','OR','97827','USA','(503) 555-2376'),
('HUNGO','Hungry Owl All-Night Grocers','Patricia McKenna','Sales Associate','8 Johnstown Road','Cork','Co. Cork',NULL,'Ireland','2967 3333'),
('ISLAT','Island Trading','Helen Bennett','Marketing Manager','Garden House Crowther Way','Cowes','Isle of Wight','PO31 7PJ','UK',NULL),
('KOENE','Königlich Essen','Philip Cramer','Sales Associate','Maubelstr. 90','Brandenburg',NULL,'14776','Germany',NULL),
('LACOR','La corne d''abondance','Daniel Tonini','Sales Representative','67, avenue de l''Europe','Versailles',NULL,'78000','France','30.59.85.11'),
('LAMAI','La maison d''Asie','Annette Roulet','Sales Manager','1 rue Alsace-Lorraine','Toulouse',NULL,'31000','France','61.77.61.11'),
('LAUGB','Laughing Bacchus Wine Cellars','Yoshi Tannamuri','Marketing Assistant','1900 Oak St.','Vancouver','BC','V3F 2K1','Canada','(604) 555-7293'),
('LAZYK','Lazy K Kountry Store','John Steel','Marketing Manager','12 Orchestra Terrace','Walla Walla','WA','99362','USA','(509) 555-6221'),
('LEHMS','Lehmanns Marktstand','Renate Messner','Sales Representative','Magazinweg 7','Frankfurt a.M.',NULL,'60528','Germany','069-0245874'),
('LETSS','Let''s Stop N Shop','Jaime Yorres','Owner','87 Polk St. Suite 5','San Francisco','CA','94117','USA',NULL),
('LILAS','LILA-Supermercado','Carlos González','Accounting Manager','Carrera 52 con Ave. Bolívar #65-98 Llano Largo','Barquisimeto','Lara','3508','Venezuela','(9) 331-7256'),
('LINOD','LINO-Delicateses','Felipe Izquierdo','Owner','Ave. 5 de Mayo Porlamar','I. de Margarita','Nueva Esparta','4980','Venezuela','(8) 34-93-93'),
('LONEP','Lonesome Pine Restaurant','Fran Wilson','Sales Manager','89 Chiaroscuro Rd.','Portland','OR','97219','USA','(503) 555-9646'),
('MAGAA','Magazzini Alimentari Riuniti','Giovanni Rovelli','Marketing Manager','Via Ludovico il Moro 22','Bergamo',NULL,'24100','Italy','035-640231'),
('MAISD','Maison Dewey','Catherine Dewey','Sales Agent','Rue Joseph-Bens 532','Bruxelles',NULL,'B-1180','Belgium','(02) 201 24 68'),
('MEREP','Mère Paillarde','Jean Fresnière','Marketing Assistant','43 rue St. Laurent','Montréal','Québec','H1J 1C3','Canada','(514) 555-8055'),
('MORGK','Morgenstern Gesundkost','Alexander Feuer','Marketing Assistant','Heerstr. 22','Leipzig',NULL,'04179','Germany',NULL),
('NORTS','North/South','Simon Crowther','Sales Associate','South House 300 Queensbridge','London',NULL,'SW7 1RZ','UK','(171) 555-2530'),
('OCEAN','Océano Atlántico Ltda.','Yvonne Moncada','Sales Agent','Ing. Gustavo Moncada 8585 Piso 20-A','Buenos Aires',NULL,'1010','Argentina','(1) 135-5535'),
('OLDWO','Old World Delicatessen','Rene Phillips','Sales Representative','2743 Bering St.','Anchorage','AK','99508','USA','(907) 555-2880'),
('OTTIK','Ottilies Käseladen','Henriette Pfalzheim','Owner','Mehrheimerstr. 369','Köln',NULL,'50739','Germany','0221-0765721'),
('PARIS','Paris spécialités','Marie Bertrand','Owner','265, boulevard Charonne','Paris',NULL,'75012','France','(1) 42.34.22.77'),
('PERIC','Pericles Comidas clásicas','Guillermo Fernández','Sales Representative','Calle Dr. Jorge Cash 321','México D.F.',NULL,'05033','Mexico','(5) 545-3745'),
('PICCO','Piccolo und mehr','Georg Pipps','Sales Manager','Geislweg 14','Salzburg',NULL,'5020','Austria','6562-9723'),
('PRINI','Princesa Isabel Vinhos','Isabel de Castro','Sales Representative','Estrada da saúde n. 58','Lisboa',NULL,'1756','Portugal',NULL),
('QUEDE','Que Delícia','Bernardo Batista','Accounting Manager','Rua da Panificadora, 12','Rio de Janeiro','RJ','02389-673','Brazil','(21) 555-4545'),
('QUEEN','Queen Cozinha','Lúcia Carvalho','Marketing Assistant','Alameda dos Canàrios, 891','Sao Paulo','SP','05487-020','Brazil',NULL),
('QUICK','QUICK-Stop','Horst Kloss','Accounting Manager','Taucherstraße 10','Cunewalde',NULL,'01307','Germany',NULL),
('RANCH','Rancho grande','Sergio Gutiérrez','Sales Representative','Av. del Libertador 900','Buenos Aires',NULL,'1010','Argentina','(1) 123-5556'),
('RATTC','Rattlesnake Canyon Grocery','Paula Wilson','Assistant Sales Representative','2817 Milton Dr.','Albuquerque','NM','87110','USA','(505) 555-3620'),
('REGGC','Reggiani Caseifici','Maurizio Moroni','Sales Associate','Strada Provinciale 124','Reggio Emilia',NULL,'42100','Italy','0522-556722'),
('RICAR','Ricardo Adocicados','Janete Limeira','Assistant Sales Agent','Av. Copacabana, 267','Rio de Janeiro','RJ','02389-890','Brazil',NULL),
('RICSU','Richter Supermarkt','Michael Holz','Sales Manager','Grenzacherweg 237','Genève',NULL,'1203','Switzerland',NULL),
('ROMEY','Romero y tomillo','Alejandra Camino','Accounting Manager','Gran Vía, 1','Madrid',NULL,'28001','Spain','(91) 745 6210'),
('SANTG','Santé Gourmet','Jonas Bergulfsen','Owner','Erling Skakkes gate 78','Stavern',NULL,'4110','Norway','07-98 92 47'),
('SAVEA','Save-a-lot Markets','Jose Pavarotti','Sales Representative','187 Suffolk Ln.','Boise','ID','83720','USA',NULL),
('SEVES','Seven Seas Imports','Hari Kumar','Sales Manager','90 Wadhurst Rd.','London',NULL,'OX15 4NB','UK','(171) 555-5646'),
('SIMOB','Simons bistro','Jytte Petersen','Owner','Vinbæltet 34','Kobenhavn',NULL,'1734','Denmark','31 13 35 57'),
('SPECD','Spécialités du monde','Dominique Perrier','Marketing Manager','25, rue Lauriston','Paris',NULL,'75016','France','(1) 47.55.60.20'),
('SPLIR','Split Rail Beer & Ale','Art Braunschweiger','Sales Manager','P.O. Box 555','Lander','WY','82520','USA','(307) 555-6525'),
('SUPRD','Suprêmes délices','Pascale Cartrain','Accounting Manager','Boulevard Tirou, 255','Charleroi',NULL,'B-6000','Belgium','(071) 23 67 22 21'),
('THEBI','The Big Cheese','Liz Nixon','Marketing Manager','89 Jefferson Way Suite 2','Portland','OR','97201','USA',NULL),
('THECR','The Cracker Box','Liu Wong','Marketing Assistant','55 Grizzly Peak Rd.','Butte','MT','59801','USA','(406) 555-8083'),
('TOMSP','Toms Spezialitäten','Karin Josephs','Marketing Manager','Luisenstr. 48','Münster',NULL,'44087','Germany','0251-035695'),
('TORTU','Tortuga Restaurante','Miguel Angel Paolino','Owner','Avda. Azteca 123','México D.F.',NULL,'05033','Mexico',NULL),
('TRADH','Tradição Hipermercados','Anabela Domingues','Sales Representative','Av. Inês de Castro, 414','Sao Paulo','SP','05634-030','Brazil','(11) 555-2168'),
('TRAIH','Trail''s Head Gourmet Provisioners','Helvetius Nagy','Sales Associate','722 DaVinci Blvd.','Kirkland','WA','98034','USA','(206) 555-2174'),
('VAFFE','Vaffeljernet','Palle Ibsen','Sales Manager','Smagsloget 45','Århus',NULL,'8200','Denmark','86 22 33 44'),
('VICTE','Victuailles en stock','Mary Saveley','Sales Agent','2, rue du Commerce','Lyon',NULL,'69004','France','78.32.54.87'),
('VINET','Vins et alcools Chevalier','Paul Henriot','Accounting Manager','59 rue de l''Abbaye','Reims',NULL,'51100','France','26.47.15.11'),
('WANDK','Die Wandernde Kuh','Rita Müller','Sales Representative','Adenauerallee 900','Stuttgart',NULL,'70563','Germany','0711-035428'),
('WARTH','Wartian Herkku','Pirkko Koskitalo','Accounting Manager','Torikatu 38','Oulu',NULL,'90110','Finland','981-443655'),
('WELLI','Wellington Importadora','Paula Parente','Sales Manager','Rua do Mercado, 12','Resende','SP','08737-363','Brazil',NULL),
('WHITC','White Clover Markets','Karl Jablonski','Owner','305 - 14th Ave. S. Suite 3B','Seattle','WA','98128','USA','(206) 555-4115'),
('WILMK','Wilman Kala','Matti Karttunen','Owner/Marketing Assistant','Keskuskatu 45','Helsinki',NULL,'21240','Finland','90-224 8858'),
('WOLZA','Wolski Zajazd','Zbyszek Piestrzeniewicz','Owner','ul. Filtrowa 68','Warszawa',NULL,'01-012','Poland','(26) 642-7012');


INSERT INTO categories (
  category_id, category_name, description
) VALUES
(1,'Beverages','Soft drinks, coffees, teas, beers, and ales'),
(2,'Condiments','Sweet and savory sauces, relishes, spreads, and seasonings'),
(3,'Confections','Desserts, candies, and sweet breads'),
(4,'Dairy Products','Cheeses'),
(5,'Grains/Cereals','Breads, crackers, pasta, and cereal'),
(6,'Meat/Poultry','Prepared meats'),
(7,'Produce','Dried fruit and bean curd'),
(8,'Seafood','Seaweed and fish');


INSERT INTO suppliers (
  supplier_id, company_name, address, city, region, postal_code, country
) VALUES
(1,'Exotic Liquids','49 Gilbert St.','London',NULL,'EC1 4SD','UK'),
(2,'New Orleans Cajun Delights','P.O. Box 78934','New Orleans','LA','70117','USA'),
(3,'Grandma Kelly''s Homestead','707 Oxford Rd.','Ann Arbor','MI','48104','USA'),
(4,'Tokyo Traders','9-8 Sekimai Musashino-shi','Tokyo',NULL,'100','Japan'),
(5,'Cooperativa de Quesos ''Las Cabras''','Calle del Rosal 4','Oviedo','Asturias','33007','Spain'),
(6,'Mayumi''s','92 Setsuko Chuo-ku','Osaka',NULL,'545','Japan'),
(7,'Pavlova, Ltd.','74 Rose St. Moonie Ponds','Melbourne','Victoria','3058','Australia'),
(8,'Specialty Biscuits, Ltd.','29 King''s Way','Manchester',NULL,'M14 GSD','UK'),
(9,'PB Knäckebröd AB','Kaloadagatan 13','Göteborg',NULL,'S-345 67','Sweden'),
(10,'Refrescos Americanas LTDA','Av. das Americanas 12.890','Sao Paulo',NULL,'5442','Brazil'),
(11,'Heli Süßwaren GmbH & Co. KG','Tiergartenstraße 5','Berlin',NULL,'10785','Germany'),
(12,'Plutzer Lebensmittelgroßmärkte AG','Bogenallee 51','Frankfurt',NULL,'60439','Germany'),
(13,'Nord-Ost-Fisch Handelsgesellschaft mbH','Frahmredder 112a','Cuxhaven',NULL,'27478','Germany'),
(14,'Formaggi Fortini s.r.l.','Viale Dante, 75','Ravenna',NULL,'48100','Italy'),
(15,'Norske Meierier','Hatlevegen 5','Sandvika',NULL,'1320','Norway'),
(16,'Bigfoot Breweries','3400 - 8th Avenue Suite 210','Bend','OR','97101','USA'),
(17,'Svensk Sjöföda AB','Brovallavägen 231','Stockholm',NULL,'S-123 45','Sweden'),
(18,'Aux joyeux ecclésiastiques','203, Rue des Francs-Bourgeois','Paris',NULL,'75004','France'),
(19,'New England Seafood Cannery','Order Processing Dept. 2100 Paul Revere Blvd.','Boston','MA','02134','USA'),
(20,'Leka Trading','471 Serangoon Loop, Suite #402','Singapore',NULL,'0512','Singapore'),
(21,'Lyngbysild','Lyngbysild Fiskebakken 10','Lyngby',NULL,'2800','Denmark'),
(22,'Zaanse Snoepfabriek','Verkoop Rijnweg 22','Zaandam',NULL,'9999 ZZ','Netherlands'),
(23,'Karkki Oy','Valtakatu 12','Lappeenranta',NULL,'53120','Finland'),
(24,'G''day, Mate','170 Prince Edward Parade Hunter''s Hill','Sydney','NSW','2042','Australia'),
(25,'Ma Maison','2960 Rue St. Laurent','Montréal','Québec','H1J 1C3','Canada'),
(26,'Pasta Buttini s.r.l.','Via dei Gelsomini, 153','Salerno',NULL,'84100','Italy'),
(27,'Escargots Nouveaux','22, rue H. Voiron','Montceau',NULL,'71300','France'),
(28,'Gai pâturage','Bat. B 3, rue des Alpes','Annecy',NULL,'74000','France'),
(29,'Forêts d''érables','148 rue Chasseur','Ste-Hyacinthe','Québec','J2S 7S8','Canada');


INSERT INTO products (
  product_id, product_name, supplier_id, category_id, quantity_per_unit,
  unit_price, units_in_stock, units_on_order, discontinued
) VALUES
(1,'Chai',1,1,'10 boxes x 20 bags',18.00,39,0,'f'),
(2,'Chang',1,1,'24 - 12 oz bottles',19.00,17,40,'f'),
(3,'Aniseed Syrup',1,2,'12 - 550 ml bottles',10.00,13,70,'f'),
(4,'Chef Anton''s Cajun Seasoning',2,2,'48 - 6 oz jars',22.00,53,0,'f'),
(5,'Chef Anton''s Gumbo Mix',2,2,'36 boxes',21.35,0,0,'t'),
(6,'Grandma''s Boysenberry Spread',3,2,'12 - 8 oz jars',25.00,120,0,'f'),
(7,'Uncle Bob''s Organic Dried Pears',3,7,'12 - 1 lb pkgs.',30.00,15,0,'f'),
(8,'Northwoods Cranberry Sauce',3,2,'12 - 12 oz jars',40.00,6,0,'f'),
(9,'Mishi Kobe Niku',4,6,'18 - 500 g pkgs.',97.00,29,0,'t'),
(10,'Ikura',4,8,'12 - 200 ml jars',31.00,31,0,'f'),
(11,'Queso Cabrales',5,4,'1 kg pkg.',21.00,22,30,'f'),
(12,'Queso Manchego La Pastora',5,4,'10 - 500 g pkgs.',38.00,86,0,'f'),
(13,'Konbu',6,8,'2 kg box',6.00,24,0,'f'),
(14,'Tofu',6,7,'40 - 100 g pkgs.',23.25,35,0,'f'),
(15,'Genen Shouyu',6,2,'24 - 250 ml bottles',15.50,39,0,'f'),
(16,'Pavlova',7,3,'32 - 500 g boxes',17.45,29,0,'f'),
(17,'Alice Mutton',7,6,'20 - 1 kg tins',39.00,0,0,'t'),
(18,'Carnarvon Tigers',7,8,'16 kg pkg.',62.50,42,0,'f'),
(19,'Teatime Chocolate Biscuits',8,3,'10 boxes x 12 pieces',9.20,25,0,'f'),
(20,'Sir Rodney''s Marmalade',8,3,'30 gift boxes',81.00,40,0,'f'),
(21,'Sir Rodney''s Scones',8,3,'24 pkgs. x 4 pieces',10.00,3,40,'f'),
(22,'Gustaf''s Knäckebröd',9,5,'24 - 500 g pkgs.',21.00,104,0,'f'),
(23,'Tunnbröd',9,5,'12 - 250 g pkgs.',9.00,61,0,'f'),
(24,'Guaraná Fantástica',10,1,'12 - 355 ml cans',4.50,20,0,'t'),
(25,'NuNuCa Nuß-Nougat-Creme',11,3,'20 - 450 g glasses',14.00,76,0,'f'),
(26,'Gumbär Gummibärchen',11,3,'100 - 250 g bags',31.23,15,0,'f'),
(27,'Schoggi Schokolade',11,3,'100 - 100 g pieces',43.90,49,0,'f'),
(28,'Rössle Sauerkraut',12,7,'25 - 825 g cans',45.60,26,0,'t'),
(29,'Thüringer Rostbratwurst',12,6,'50 bags x 30 sausgs.',123.79,0,0,'t'),
(30,'Nord-Ost Matjeshering',13,8,'10 - 200 g glasses',25.89,10,0,'f'),
(31,'Gorgonzola Telino',14,4,'12 - 100 g pkgs',12.50,0,70,'f'),
(32,'Mascarpone Fabioli',14,4,'24 - 200 g pkgs.',32.00,9,40,'f'),
(33,'Geitost',16,4,'500 g',2.50,112,0,'f'),
(34,'Sasquatch Ale',16,1,'24 - 12 oz bottles',14.00,111,0,'f'),
(35,'Steeleye Stout',16,1,'24 - 12 oz bottles',18.00,20,0,'f'),
(36,'Inlagd Sill',17,8,'24 - 250 g jars',19.00,112,0,'f'),
(37,'Gravad lax',17,8,'12 - 500 g pkgs.',26.00,11,50,'f'),
(38,'Côte de Blaye',18,1,'12 - 75 cl bottles',263.50,17,0,'f'),
(39,'Chartreuse verte',18,1,'750 cc per bottle',18.00,69,0,'f'),
(40,'Boston Crab Meat',19,8,'24 - 4 oz tins',18.40,123,0,'f'),
(41,'Jack''s New England Clam Chowder',19,8,'12 - 12 oz cans',9.65,85,0,'f'),
(42,'Singaporean Hokkien Fried Mee',20,5,'32 - 1 kg pkgs.',14.00,26,0,'t'),
(43,'Ipoh Coffee',20,1,'16 - 500 g tins',46.00,17,10,'f'),
(44,'Gula Malacca',20,2,'20 - 2 kg bags',19.45,27,0,'f'),
(45,'Rogede sild',21,8,'1k pkg.',9.50,5,70,'f'),
(46,'Spegesild',21,8,'4 - 450 g glasses',12.00,95,0,'f'),
(47,'Zaanse koeken',22,3,'10 - 4 oz boxes',9.50,36,0,'f'),
(48,'Chocolade',22,3,'10 pkgs.',12.75,15,70,'f'),
(49,'Maxilaku',23,3,'24 - 50 g pkgs.',20.00,10,60,'f'),
(50,'Valkoinen suklaa',23,3,'12 - 100 g bars',16.25,65,0,'f'),
(51,'Manjimup Dried Apples',24,7,'50 - 300 g pkgs.',53.00,20,0,'f'),
(52,'Filo Mix',24,5,'16 - 2 kg boxes',7.00,38,0,'f'),
(53,'Perth Pasties',24,6,'48 pieces',32.80,0,0,'t'),
(54,'Tourtière',25,6,'16 pies',7.45,21,0,'f'),
(55,'Pâté chinois',25,6,'24 boxes x 2 pies',24.00,115,0,'f'),
(56,'Gnocchi di nonna Alice',26,5,'24 - 250 g pkgs.',38.00,21,10,'f'),
(57,'Ravioli Angelo',26,5,'24 - 250 g pkgs.',19.50,36,0,'f'),
(58,'Escargots de Bourgogne',27,8,'24 pieces',13.25,62,0,'f'),
(59,'Raclette Courdavault',28,4,'5 kg pkg.',55.00,79,0,'f'),
(60,'Camembert Pierrot',28,4,'15 - 300 g rounds',34.00,19,0,'f'),
(61,'Sirop d''érable',29,2,'24 - 500 ml bottles',28.50,113,0,'f'),
(62,'Tarte au sucre',29,3,'48 pies',49.30,17,0,'f'),
(63,'Vegie-spread',7,2,'15 - 625 g jars',43.90,24,0,'f'),
(64,'Wimmers gute Semmelknödel',12,5,'20 bags x 4 pieces',33.25,22,80,'f'),
(65,'Louisiana Fiery Hot Pepper Sauce',2,2,'32 - 8 oz bottles',21.05,76,0,'f'),
(66,'Louisiana Hot Spiced Okra',2,2,'24 - 8 oz jars',17.00,4,100,'f'),
(67,'Laughing Lumberjack Lager',16,1,'24 - 12 oz bottles',14.00,52,0,'f'),
(68,'Scottish Longbreads',8,3,'10 boxes x 8 pieces',12.50,6,10,'f'),
(69,'Gudbrandsdalsost',16,4,'10 kg pkg.',36.00,26,0,'f'),
(70,'Outback Lager',7,1,'24 - 355 ml bottles',15.00,15,10,'f'),
(71,'Flotemysost',16,4,'10 - 500 g pkgs.',21.50,26,0,'f'),
(72,'Mozzarella di Giovanni',14,4,'24 - 200 g pkgs.',34.80,14,0,'f'),
(73,'Röd Kaviar',17,8,'24 - 150 g jars',15.00,101,0,'f'),
(74,'Longlife Tofu',4,7,'5 kg pkg.',10.00,4,20,'f'),
(75,'Rhönbräu Klosterbier',12,1,'24 - 0.5 l bottles',7.75,125,0,'f'),
(76,'Lakkalikööri',23,1,'500 ml',18.00,57,0,'f'),
(77,'Original Frankfurter grüne Soße',12,2,'12 boxes',13.00,32,0,'f');

INSERT INTO orders (
  order_id, customer_id, employee_id, order_date, shipped_date, ship_via, freight,
  ship_address, ship_city, ship_region, ship_postal_code, ship_country
) VALUES
(10248,'VINET',5,'2016-07-04','2016-07-16',3,32.38,'59 rue de l''Abbaye','Reims',NULL,'51100','France'),
(10249,'TOMSP',6,'2016-07-05','2016-07-10',1,11.61,'Luisenstr. 48','Münster',NULL,'44087','Germany'),
(10250,'HANAR',4,'2016-07-08','2016-07-12',2,65.83,'Rua do Paço, 67','Rio de Janeiro','RJ','05454-876','Brazil'),
(10251,'VICTE',3,'2016-07-08','2016-07-15',1,41.34,'2, rue du Commerce','Lyon',NULL,'69004','France'),
(10252,'SUPRD',4,'2016-07-09','2016-07-11',2,51.30,'Boulevard Tirou, 255','Charleroi',NULL,'B-6000','Belgium'),
(10253,'HANAR',3,'2016-07-10','2016-07-16',2,58.17,'Rua do Paço, 67','Rio de Janeiro','RJ','05454-876','Brazil'),
(10254,'CHOPS',5,'2016-07-11','2016-07-23',2,22.98,'Hauptstr. 31','Bern',NULL,'3012','Switzerland'),
(10255,'RICSU',9,'2016-07-12','2016-07-15',3,148.33,'Starenweg 5','Genève',NULL,'1204','Switzerland'),
(10256,'WELLI',3,'2016-07-15','2016-07-17',2,13.97,'Rua do Mercado, 12','Resende','SP','08737-363','Brazil'),
(10257,'HILAA',10,'2016-07-16','2016-07-22',3,81.91,'Carrera 22 con Ave. Carlos Soublette #8-35','San Cristóbal','Táchira','5022','Venezuela'),
(10258,'ERNSH',1,'2016-07-17','2016-07-23',1,140.51,'Kirchgasse 6','Graz',NULL,'8010','Austria'),
(10259,'CENTC',4,'2016-07-18','2016-07-25',3,3.25,'Sierras de Granada 9993','México D.F.',NULL,'05022','Mexico'),
(10260,'OTTIK',4,'2016-07-19','2016-07-29',1,55.09,'Mehrheimerstr. 369','Köln',NULL,'50739','Germany'),
(10261,'QUEDE',4,'2016-07-19','2016-07-30',2,3.05,'Rua da Panificadora, 12','Rio de Janeiro','RJ','02389-673','Brazil'),
(10262,'RATTC',8,'2016-07-22','2016-07-25',3,48.29,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10263,'ERNSH',9,'2016-07-23','2016-07-31',3,146.06,'Kirchgasse 6','Graz',NULL,'8010','Austria'),
(10264,'FOLKO',6,'2016-07-24','2016-08-23',3,3.67,'Åkergatan 24','Bräcke',NULL,'S-844 67','Sweden'),
(10265,'BLONP',2,'2016-07-25','2016-08-12',1,55.28,'24, place Kléber','Strasbourg',NULL,'67000','France'),
(10266,'WARTH',3,'2016-07-26','2016-07-31',3,25.73,'Torikatu 38','Oulu',NULL,'90110','Finland'),
(10267,'FRANK',4,'2016-07-29','2016-08-06',1,208.58,'Berliner Platz 43','München',NULL,'80805','Germany'),
(10268,'GROSR',8,'2016-07-30','2016-08-02',3,66.29,'5ª Ave. Los Palos Grandes','Caracas','DF','1081','Venezuela'),
(10269,'WHITC',5,'2016-07-31','2016-08-09',1,4.56,'1029 - 12th Ave. S.','Seattle','WA','98124','USA'),
(10270,'WARTH',1,'2016-08-01','2016-08-02',1,136.54,'Torikatu 38','Oulu',NULL,'90110','Finland'),
(10271,'SPLIR',6,'2016-08-01','2016-08-30',2,4.54,'P.O. Box 555','Lander','WY','82520','USA'),
(10272,'RATTC',6,'2016-08-02','2016-08-06',2,98.03,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10273,'QUICK',3,'2016-08-05','2016-08-12',3,76.07,'Taucherstraße 10','Cunewalde',NULL,'01307','Germany'),
(10274,'VINET',6,'2016-08-06','2016-08-16',1,6.01,'59 rue de l''Abbaye','Reims',NULL,'51100','France'),
(10275,'MAGAA',1,'2016-08-07','2016-08-09',1,26.93,'Via Ludovico il Moro 22','Bergamo',NULL,'24100','Italy'),
(10276,'TORTU',8,'2016-08-08','2016-08-14',3,13.84,'Avda. Azteca 123','México D.F.',NULL,'05033','Mexico'),
(10277,'MORGK',2,'2016-08-09','2016-08-13',3,125.77,'Heerstr. 22','Leipzig',NULL,'04179','Germany'),
(10278,'BERGS',8,'2016-08-12','2016-08-16',2,92.69,'Berguvsvägen 8','Luleå',NULL,'S-958 22','Sweden'),
(10279,'LEHMS',8,'2016-08-13','2016-08-16',2,25.83,'Magazinweg 7','Frankfurt a.M.',NULL,'60528','Germany'),
(10280,'BERGS',2,'2016-08-14','2016-09-12',1,8.98,'Berguvsvägen 8','Luleå',NULL,'S-958 22','Sweden'),
(10281,'ROMEY',4,'2016-08-14','2016-08-21',1,2.94,'Gran Vía, 1','Madrid',NULL,'28001','Spain'),
(10282,'ROMEY',4,'2016-08-15','2016-08-21',1,12.69,'Gran Vía, 1','Madrid',NULL,'28001','Spain'),
(10283,'LILAS',3,'2016-08-16','2016-08-23',3,84.81,'Carrera 52 con Ave. Bolívar #65-98 Llano Largo','Barquisimeto','Lara','3508','Venezuela'),
(10284,'LEHMS',4,'2016-08-19','2016-08-27',1,76.56,'Magazinweg 7','Frankfurt a.M.',NULL,'60528','Germany'),
(10285,'QUICK',1,'2016-08-20','2016-08-26',2,76.83,'Taucherstraße 10','Cunewalde',NULL,'01307','Germany'),
(10286,'QUICK',8,'2016-08-21','2016-08-30',3,229.24,'Taucherstraße 10','Cunewalde',NULL,'01307','Germany'),
(10287,'RICAR',8,'2016-08-22','2016-08-28',3,12.76,'Av. Copacabana, 267','Rio de Janeiro','RJ','02389-890','Brazil'),
(10288,'REGGC',4,'2016-08-23','2016-09-03',1,7.45,'Strada Provinciale 124','Reggio Emilia',NULL,'42100','Italy'),
(10289,'BSBEV',7,'2016-08-26','2016-08-28',3,22.77,'Fauntleroy Circus','London',NULL,'EC2 5NT','UK'),
(10290,'COMMI',8,'2016-08-27','2016-09-03',1,79.70,'Av. dos Lusíadas, 23','Sao Paulo','SP','05432-043','Brazil'),
(10291,'QUEDE',6,'2016-08-27','2016-09-04',2,6.40,'Rua da Panificadora, 12','Rio de Janeiro','RJ','02389-673','Brazil'),
(10292,'TRADH',1,'2016-08-28','2016-09-02',2,1.35,'Av. Inês de Castro, 414','Sao Paulo','SP','05634-030','Brazil'),
(10293,'TORTU',1,'2016-08-29','2016-09-11',3,21.18,'Avda. Azteca 123','México D.F.',NULL,'05033','Mexico'),
(10294,'RATTC',4,'2016-08-30','2016-09-05',2,147.26,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10295,'VINET',2,'2016-09-02','2016-09-10',2,1.15,'59 rue de l''Abbaye','Reims',NULL,'51100','France'),
(10296,'LILAS',6,'2016-09-03','2016-09-11',1,0.12,'Carrera 52 con Ave. Bolívar #65-98 Llano Largo','Barquisimeto','Lara','3508','Venezuela'),
(10297,'BLONP',5,'2016-09-04','2016-09-10',2,5.74,'24, place Kléber','Strasbourg',NULL,'67000','France'),
(10298,'HUNGO',6,'2016-09-05','2016-09-11',2,168.22,'8 Johnstown Road','Cork','Co. Cork',NULL,'Ireland'),
(10299,'RICAR',4,'2016-09-06','2016-09-13',2,29.76,'Av. Copacabana, 267','Rio de Janeiro','RJ','02389-890','Brazil'),
(10300,'MAGAA',2,'2016-09-09','2016-09-18',2,17.68,'Via Ludovico il Moro 22','Bergamo',NULL,'24100','Italy'),
(10301,'WANDK',8,'2016-09-09','2016-09-17',2,45.08,'Adenauerallee 900','Stuttgart',NULL,'70563','Germany'),
(10302,'SUPRD',4,'2016-09-10','2016-10-09',2,6.27,'Boulevard Tirou, 255','Charleroi',NULL,'B-6000','Belgium'),
(10303,'GODOS',7,'2016-09-11','2016-09-18',2,107.83,'C/ Romero, 33','Sevilla',NULL,'41101','Spain'),
(10304,'TORTU',1,'2016-09-12','2016-09-17',2,63.79,'Avda. Azteca 123','México D.F.',NULL,'05033','Mexico'),
(10305,'OLDWO',8,'2016-09-13','2016-10-09',3,257.62,'2743 Bering St.','Anchorage','AK','99508','USA'),
(10306,'ROMEY',1,'2016-09-16','2016-09-23',3,7.56,'Gran Vía, 1','Madrid',NULL,'28001','Spain'),
(10307,'LONEP',2,'2016-09-17','2016-09-25',2,0.56,'89 Chiaroscuro Rd.','Portland','OR','97219','USA'),
(10308,'ANATR',7,'2016-09-18','2016-09-24',3,1.61,'Avda. de la Constitución 2222','México D.F.',NULL,'05021','Mexico'),
(10309,'HUNGO',3,'2016-09-19','2016-10-23',1,47.30,'8 Johnstown Road','Cork','Co. Cork',NULL,'Ireland'),
(10310,'THEBI',8,'2016-09-20','2016-09-27',2,17.52,'89 Jefferson Way Suite 2','Portland','OR','97201','USA'),
(10311,'DUMON',1,'2016-09-20','2016-09-26',3,24.69,'67, rue des Cinquante Otages','Nantes',NULL,'44000','France'),
(10312,'WANDK',2,'2016-09-23','2016-10-03',2,40.26,'Adenauerallee 900','Stuttgart',NULL,'70563','Germany'),
(10313,'QUICK',2,'2016-09-24','2016-10-04',2,1.96,'Taucherstraße 10','Cunewalde',NULL,'01307','Germany'),
(10314,'RATTC',1,'2016-09-25','2016-10-04',2,74.16,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10315,'ISLAT',4,'2016-09-26','2016-10-03',2,41.76,'Garden House Crowther Way','Cowes','Isle of Wight','PO31 7PJ','UK'),
(10316,'RATTC',1,'2016-09-27','2016-10-08',3,150.15,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10317,'LONEP',6,'2016-09-30','2016-10-10',1,12.69,'89 Chiaroscuro Rd.','Portland','OR','97219','USA'),
(10318,'ISLAT',8,'2016-10-01','2016-10-04',2,4.73,'Garden House Crowther Way','Cowes','Isle of Wight','PO31 7PJ','UK'),
(10319,'TORTU',7,'2016-10-02','2016-10-11',3,64.50,'Avda. Azteca 123','México D.F.',NULL,'05033','Mexico'),
(10320,'WARTH',5,'2016-10-03','2016-10-18',3,34.57,'Torikatu 38','Oulu',NULL,'90110','Finland'),
(10321,'ISLAT',3,'2016-10-03','2016-10-11',2,3.43,'Garden House Crowther Way','Cowes','Isle of Wight','PO31 7PJ','UK'),
(10322,'PERIC',7,'2016-10-04','2016-10-23',3,0.40,'Calle Dr. Jorge Cash 321','México D.F.',NULL,'05033','Mexico'),
(10323,'KOENE',4,'2016-10-07','2016-10-14',1,4.88,'Maubelstr. 90','Brandenburg',NULL,'14776','Germany'),
(10324,'SAVEA',9,'2016-10-08','2016-10-10',1,214.27,'187 Suffolk Ln.','Boise','ID','83720','USA'),
(10325,'KOENE',1,'2016-10-09','2016-10-14',3,64.86,'Maubelstr. 90','Brandenburg',NULL,'14776','Germany'),
(10326,'BOLID',4,'2016-10-10','2016-10-14',2,77.92,'C/ Araquil, 67','Madrid',NULL,'28023','Spain'),
(10327,'FOLKO',2,'2016-10-11','2016-10-14',1,63.36,'Åkergatan 24','Bräcke',NULL,'S-844 67','Sweden'),
(10328,'FURIB',4,'2016-10-14','2016-10-17',3,87.03,'Jardim das rosas n. 32','Lisboa',NULL,'1675','Portugal'),
(10329,'SPLIR',4,'2016-10-15','2016-10-23',2,191.67,'P.O. Box 555','Lander','WY','82520','USA'),
(10330,'LILAS',3,'2016-10-16','2016-10-28',1,12.75,'Carrera 52 con Ave. Bolívar #65-98 Llano Largo','Barquisimeto','Lara','3508','Venezuela'),
(10331,'BONAP',9,'2016-10-16','2016-10-21',1,10.19,'12, rue des Bouchers','Marseille',NULL,'13008','France'),
(10332,'MEREP',3,'2016-10-17','2016-10-21',2,52.84,'43 rue St. Laurent','Montréal','Québec','H1J 1C3','Canada'),
(10333,'WARTH',5,'2016-10-18','2016-10-25',3,0.59,'Torikatu 38','Oulu',NULL,'90110','Finland'),
(10334,'VICTE',8,'2016-10-21','2016-10-28',2,8.56,'2, rue du Commerce','Lyon',NULL,'69004','France'),
(10335,'HUNGO',7,'2016-10-22','2016-10-24',2,42.11,'8 Johnstown Road','Cork','Co. Cork',NULL,'Ireland'),
(10336,'PRINI',7,'2016-10-23','2016-10-25',2,15.51,'Estrada da saúde n. 58','Lisboa',NULL,'1756','Portugal'),
(10337,'FRANK',4,'2016-10-24','2016-10-29',3,108.26,'Berliner Platz 43','München',NULL,'80805','Germany'),
(10338,'OLDWO',4,'2016-10-25','2016-10-29',3,84.21,'2743 Bering St.','Anchorage','AK','99508','USA'),
(10339,'MEREP',2,'2016-10-28','2016-11-04',2,15.66,'43 rue St. Laurent','Montréal','Québec','H1J 1C3','Canada'),
(10340,'BONAP',1,'2016-10-29','2016-11-08',3,166.31,'12, rue des Bouchers','Marseille',NULL,'13008','France'),
(10341,'SIMOB',7,'2016-10-29','2016-11-05',3,26.78,'Vinbæltet 34','Kobenhavn',NULL,'1734','Denmark'),
(10342,'FRANK',4,'2016-10-30','2016-11-04',2,54.83,'Berliner Platz 43','München',NULL,'80805','Germany'),
(10343,'LEHMS',4,'2016-10-31','2016-11-06',1,110.37,'Magazinweg 7','Frankfurt a.M.',NULL,'60528','Germany'),
(10344,'WHITC',4,'2016-11-01','2016-11-05',2,23.29,'1029 - 12th Ave. S.','Seattle','WA','98124','USA'),
(10345,'QUICK',2,'2016-11-04','2016-11-11',2,249.06,'Taucherstraße 10','Cunewalde',NULL,'01307','Germany'),
(10346,'RATTC',3,'2016-11-05','2016-11-08',3,142.08,'2817 Milton Dr.','Albuquerque','NM','87110','USA'),
(10347,'FAMIA',4,'2016-11-06','2016-11-08',3,3.10,'Rua Orós, 92','Sao Paulo','SP','05442-030','Brazil');

INSERT INTO order_items (order_id, product_id, unit_price, quantity, discount) VALUES
(10248,11,14.00,12,0.00),
(10248,42,9.80,10,0.00),
(10248,72,34.80,5,0.00),
(10249,14,18.60,9,0.00),
(10249,51,42.40,40,0.00),
(10250,41,7.70,10,0.00),
(10250,51,42.40,35,0.15),
(10250,65,16.80,15,0.15),
(10251,22,16.80,6,0.05),
(10251,57,15.60,15,0.05),
(10251,65,16.80,20,0.00),
(10252,20,64.80,40,0.05),
(10252,33,2.00,25,0.05),
(10252,60,27.20,40,0.00),
(10253,31,10.00,20,0.00),
(10253,39,14.40,42,0.00),
(10253,49,16.00,40,0.00),
(10254,24,3.60,15,0.15),
(10254,55,19.20,21,0.15),
(10254,74,8.00,21,0.00),
(10255,2,15.20,20,0.00),
(10255,16,13.90,35,0.00),
(10255,36,15.20,25,0.00),
(10255,59,44.00,30,0.00),
(10256,53,26.20,15,0.00),
(10256,77,10.40,12,0.00),
(10257,27,35.10,25,0.00),
(10257,39,14.40,6,0.00),
(10257,77,10.40,15,0.00),
(10258,2,15.20,50,0.20),
(10258,5,17.00,65,0.20),
(10258,32,25.60,6,0.20),
(10259,21,8.00,10,0.00),
(10259,37,20.80,1,0.00),
(10260,41,7.70,16,0.25),
(10260,57,15.60,50,0.00),
(10260,62,39.40,15,0.25),
(10260,70,12.00,21,0.25),
(10261,21,8.00,20,0.00),
(10261,35,14.40,20,0.00),
(10262,5,17.00,12,0.20),
(10262,7,24.00,15,0.00),
(10262,56,30.40,2,0.00),
(10263,16,13.90,60,0.25),
(10263,24,3.60,28,0.00),
(10263,30,20.70,60,0.25),
(10263,74,8.00,36,0.25),
(10264,2,15.20,35,0.00),
(10264,41,7.70,25,0.15),
(10265,17,31.20,30,0.00),
(10265,70,12.00,20,0.00),
(10266,12,30.40,12,0.05),
(10267,40,14.70,50,0.00),
(10267,59,44.00,70,0.15),
(10267,76,14.40,15,0.15),
(10268,29,99.00,10,0.00),
(10268,72,27.80,4,0.00),
(10269,33,2.00,60,0.05),
(10269,72,27.80,20,0.05),
(10270,36,15.20,30,0.00),
(10270,43,36.80,25,0.00),
(10271,33,2.00,24,0.00),
(10272,20,64.80,6,0.00),
(10272,31,10.00,40,0.00),
(10272,72,27.80,24,0.00),
(10273,10,24.80,24,0.05),
(10273,31,10.00,15,0.05),
(10273,33,2.00,20,0.00),
(10273,40,14.70,60,0.05),
(10273,76,14.40,33,0.05),
(10274,71,17.20,20,0.00),
(10274,72,27.80,7,0.00),
(10275,24,3.60,12,0.05),
(10275,59,44.00,6,0.05),
(10276,10,24.80,15,0.00),
(10276,13,4.80,10,0.00),
(10277,28,36.40,20,0.00),
(10277,62,39.40,12,0.00),
(10278,44,15.50,16,0.00),
(10278,59,44.00,15,0.00),
(10278,63,35.10,8,0.00),
(10278,73,12.00,25,0.00),
(10279,17,31.20,15,0.25),
(10280,24,3.60,12,0.00),
(10280,55,19.20,20,0.00),
(10280,75,6.20,30,0.00),
(10281,19,7.30,1,0.00),
(10281,24,3.60,6,0.00),
(10281,35,14.40,4,0.00),
(10282,30,20.70,6,0.00),
(10282,57,15.60,2,0.00),
(10283,15,12.40,20,0.00),
(10283,19,7.30,18,0.00),
(10283,60,27.20,35,0.00),
(10283,72,27.80,3,0.00),
(10284,27,35.10,15,0.25),
(10284,44,15.50,21,0.00),
(10284,60,27.20,20,0.25),
(10284,67,11.20,5,0.25),
(10285,1,14.40,45,0.20);
""")
print(f"Database ready ✅ ({DB_FILE})")


Database ready ✅ (class.db)


## The Northwind database          

Throughout this notebook, we’ll work with **Northwind**, Microsoft’s classic sample database for SQL Server.

Northwind represents a fictional company and includes realistic data about:
- customers
- suppliers
- products
- employees
- orders

It’s widely used for learning SQL because it mirrors real business scenarios.

For this course, the database has been **slightly modified** to better support the topics and exercises we’ll cover.

In the upcoming sections, you’ll write queries that:
- combine data from multiple tables
- aggregate and summarize information
- produce meaningful SQL reports

But first, let’s get familiar with the tables we’ll be using.


In [3]:
# @title Northwind ER Diagram
%%html
<img id="er-img" style="width:80%; max-width:100%; height:auto;"
     data-light="https://raw.githubusercontent.com/Haross/DB_pics_nt/main/reporting_ER_northwind.png"
     data-dark="https://raw.githubusercontent.com/Haross/DB_pics_nt/main/reporting_ER_northwind_black.png"
     alt="ER diagram">

<script>
  const img = document.getElementById("er-img");

  function isDarkTheme() {
    // Colab sets html[theme=dark] on the top document
    const themeAttr = document.documentElement.getAttribute("theme");
    if (themeAttr) return themeAttr === "dark";

    // fallback: OS/browser preference
    return window.matchMedia && window.matchMedia("(prefers-color-scheme: dark)").matches;
  }

  function updateImage() {
    img.src = isDarkTheme() ? img.dataset.dark : img.dataset.light;
  }

  updateImage();

  // React to Colab theme toggles (attribute changes)
  new MutationObserver(updateImage).observe(document.documentElement, {
    attributes: true,
    attributeFilter: ["theme"]
  });

  // React to OS/browser theme changes (fallback)
  if (window.matchMedia) {
    const mq = window.matchMedia("(prefers-color-scheme: dark)");
    mq.addEventListener?.("change", updateImage);
    mq.addListener?.(updateImage); // older browsers
  }
</script>

### The employees table

Let’s start with the `employees` table.

This table stores information about the people employed at Northwind.

For each employee, it includes:
- a unique identifier (`employee_id`)
- first and last name (`first_name`, `last_name`)
- a professional title (`title`)

One column worth special attention is `reports_to`.

This column contains the `employee_id` of the employee’s manager, meaning it references **the same table**.  
In other words, the `employees` table has a built-in hierarchical structure.

You’ll also find additional descriptive information, such as:
- `hire_date`
- `address`

These columns won’t always be used in queries, but they help make the dataset more realistic.


In [4]:
# @title Employee table — sample preview
df = pd.read_sql("SELECT * FROM employees LIMIT 5", conn).style.format(na_rep="NULL").hide(axis="index")
df

employee_id,last_name,first_name,title,birth_date,hire_date,address,city,region,postal_code,country,reports_to
1,Davolio,Nancy,Sales Representative,1968-12-08,2012-05-01,507 - 20th Ave. E. Apt. 2A,Seattle,WA,98122,USA,2.000000
2,Fuller,Andrew,"Vice President, Sales",1972-02-19,2012-08-14,908 W. Capital Way,Tacoma,WA,98401,USA,NULL
3,Smith,John,Sales Representative,1983-08-30,2012-04-01,722 Moss Bay Blvd.,Kirkland,WA,98033,USA,2.000000
4,Peacock,Margaret,Sales Representative,1957-09-19,2013-05-03,4110 Old Redmond Rd.,Redmond,WA,98052,USA,2.000000
5,Buchanan,Steven,Sales Manager,1975-03-04,2013-10-17,14 Garrett Hill,London,NULL,SW1 8JR,UK,2.000000


### The customers table

Perfect, let’s move on to the `customers` table.

This table stores information about Northwind’s customers.

Each customer is identified by a unique `customer_id`, which is a five-letter abbreviation derived from the company name stored in `company_name`.

You’ll also find:
- details about the contact person (`contact_name`, `contact_title`)
- address-related information
- communication fields such as fax number

This table plays a central role in reporting, as many queries will analyze customer activity, orders, and purchasing behavior.


In [5]:
# @title Customers table — sample preview
df = pd.read_sql("SELECT * FROM customers LIMIT 5", conn).style.format(na_rep="NULL").hide(axis="index")
df

customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,fax
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NULL,05021,Mexico,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NULL,05023,Mexico,NULL
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NULL,WA1 1DP,UK,(171) 555-6750
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,NULL,S-958 22,Sweden,0921-12 34 67


### The products table

The `products` table stores information about the items sold by Northwind.

For each product, it includes:
- a unique identifier (`product_id`)
- the product name (`product_name`)
- the supplier (`supplier_id`)
- the category it belongs to (`category_id`)
- the unit price (`unit_price`)

There’s also a `discontinued` column, which indicates whether a product is still available (`false`) or no longer sold (`true`).


In [6]:
# @title Products table — sample preview
df = pd.read_sql("SELECT * FROM products LIMIT 5", conn).style.format(na_rep="NULL").hide(axis="index")
df

product_id,product_name,supplier_id,category_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,discontinued
1,Chai,1,1,10 boxes x 20 bags,18.000000,39,0,f
2,Chang,1,1,24 - 12 oz bottles,19.000000,17,40,f
3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.000000,13,70,f
4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.000000,53,0,f
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.350000,0,0,t



### The categories table

Products are organized into categories, which are stored in the `categories` table.

Each category has:
- a unique identifier
- a `category_name`
- a short description

---

### How they work together

Because the `products` table contains `category_id`, you can easily join it with the `categories` table.

This allows you to answer questions like:
- Which products belong to each category?
- What is the average price per category?
- How many discontinued products are in each category?

You’ll be using this relationship frequently when building reports.


In [7]:
# @title Example 1
base_example_1 = make_df_validator_nospoilers(
    expected_hash='720b70b91a9a62fcf67f9577745beaf8dd44f2258b3eadfed0cafeb03a4bf3a0',
    required_cols=['product_name', 'category_name', 'quantity_per_unit', 'unit_price', 'units_in_stock'],
    expected_rows=77,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_example_1 = base_example_1

make_sql_runner(
    conn,
    runner_id="example_1",
    description_md='### Example 1\nFor each product, display its name (product_name), the name of the category it belongs to (category_name), quantity per unit (quantity_per_unit), the unit price (unit_price), and the number of units in stock (units_in_stock). Order the results by unit price.\n',
    validator=val_example_1,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'categories'],
    default_sql='SELECT  product_name, category_name, quantity_per_unit, unit_price, units_in_stock\nFROM products\nJOIN categories\n  ON products.category_id = categories.category_id\nORDER BY unit_price;\n'
)


### The suppliers table

Good job, next up is the `suppliers` table.

This table is very similar in structure to the `customers` table.

Each supplier has:
- a unique identifier (`supplier_id`)
- a company name (`company_name`)
- contact-related information (such as contact name, address, and phone details)

Suppliers are linked to products through the `supplier_id` column in the `products` table.

This relationship allows you to analyze things like:
- which suppliers provide which products
- how many products each supplier offers
- pricing patterns by supplier


In [8]:
# @title Example 2
base_example_2 = make_df_validator_nospoilers(
    expected_hash='0180f1be0c5db6337718604c3a4b91c837a9e9e6e668c4ea75a0d3b39f96edaf',
    required_cols=['supplier_id', 'company_name', 'products_count'],
    expected_rows=5,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_example_2 = base_example_2

make_sql_runner(
    conn,
    runner_id="example_2",
    description_md="### Example 2\nWe'd like to see information about all the suppliers who provide the store four or more different products. Show the following columns: supplier_id, company_name, and products_count (the number of products supplied).\n",
    validator=val_example_2,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'suppliers'],
    default_sql='SELECT s.supplier_id, s.company_name, COUNT(*) AS products_count\nFROM products p\nJOIN suppliers s  ON p.supplier_id = s.supplier_id\nGROUP BY s.supplier_id, s.company_name\nHAVING COUNT(*) > 3;\n'
)


### The orders table

The `orders` table stores **general information about each order**.

It includes:
- a unique order identifier (`order_id`)
- the customer who placed the order (`customer_id`)
- the employee who handled the sale (`employee_id`)
- timestamps such as `order_date` and `shipped_date`
- additional columns related to the shipping process

Each row represents **one order as a whole**.

---

### The order_items table

The `order_items` table contains the **detailed contents of each order**.

Each row represents **one product within an order**, and includes:
- the order it belongs to (`order_id`)
- the product being sold (`product_id`)
- the unit price
- the quantity ordered
- an optional discount

---

### How they work together

An order can contain **many items**, but each order item belongs to **one order**.

This one-to-many relationship is central to reporting tasks such as:
- calculating total order values
- analyzing product sales
- building customer or employee performance reports


In [9]:
# @title Example 3
base_example_3 = make_df_validator_nospoilers(
    expected_hash='3719186546268fb432c614c9e0bac89d336ea44f956532cea27908e3894a2748',
    required_cols=['product_name', 'quantity', 'unit_price', 'discount', 'order_date'],
    expected_rows=3,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_example_3 = base_example_3

make_sql_runner(
    conn,
    runner_id="example_3",
    description_md='### Example 3\nDisplay the list of products purchased in the order with ID equal to **10250**. Show the following information: product name (product_name), the quantity of the product ordered (quantity), the unit price (unit_price from the order_items table), the discount (discount), and the order_date. Order the items by product name.\n',
    validator=val_example_3,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['order_items', 'products', 'orders'],
    default_sql='SELECT product_name, quantity, order_items.unit_price, discount, order_date\nFROM order_items\nJOIN products  ON order_items.product_id = products.product_id\nJOIN orders ON orders.order_id = order_items.order_id\nWHERE orders.order_id = 10250\nORDER BY product_name;\n'
)


### Summary

That’s it! You’re now familiar with all seven tables in the Northwind database.

Before moving on, let’s do a quick recap.  
Throughout this course, we’ll be working with the following tables:

- **employees** — information about Northwind’s employees  
- **customers** — details about Northwind’s customers  
- **products** — products sold by the company  
- **categories** — product categories  
- **suppliers** — companies that supply the products  
- **orders** — general information about customer orders  
- **order_items** — individual items within each order  

Understanding how these tables are structured, and how they relate to one another is essential for building meaningful SQL reports.


## Detail reports

Let’s get started.

One of the simplest and most common types of reports is a **detail report**.

Detail reports focus on individual business objects (such as orders) and include **rich, descriptive information** about them.  
Because this information is often spread across multiple tables, we need a way to bring everything together into a single result.

In SQL, we do this using `JOIN`s.

Take a look at the following example:

```sql
SELECT
  c.company_name AS customer_company_name,
  e.first_name AS employee_first_name,
  e.last_name AS employee_last_name,
  o.order_date,
  o.shipped_date,
  o.ship_country
FROM orders o
JOIN employees e
  ON o.employee_id = e.employee_id
JOIN customers c
  ON o.customer_id = c.customer_id
WHERE o.ship_country = 'France';
````

This query builds a detailed report of **orders shipped to France**.

To do that:

* we start from the `orders` table
* we join `employees` to get information about who handled each order
* we join `customers` to get information about who placed the order
* we filter the results to shipments to France

Notice that each table is given a short **alias** (`o`, `e`, `c`).

Aliases make queries shorter, easier to read, and are standard practice when working with larger SQL reports.


In [10]:
# @title Practice 1
base_practice_1 = make_df_validator_nospoilers(
    expected_hash='e56a5135e0d0042948c8c771b770fe6a94edf7cf56bdd058d9c2494d08e9246e',
    required_cols=['product_name', 'unit_price', 'quantity', 'supplier_name'],
    expected_rows=3,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_1 = base_practice_1

make_sql_runner(
    conn,
    runner_id="practice_1",
    description_md="### Practice 1\nShow the following information related to all items with **order_id = 10248**: the product name, the unit price (taken from the order_items table), the quantity, and the name of the supplier's company (as supplier_name).\n",
    validator=val_practice_1,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['order_items', 'products', 'suppliers']
)


In [11]:
# @title Practice 2
base_practice_2 = make_df_validator_nospoilers(
    expected_hash='cbb5f09037725d48b0bbb50ea00dc8ac59386b3644b25ac607b062d4667e3f26',
    required_cols=['product_name', 'company_name', 'category_name', 'unit_price', 'quantity_per_unit'],
    expected_rows=77,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_2 = base_practice_2

make_sql_runner(
    conn,
    runner_id="practice_2",
    description_md='### Practice 2\nShow the following information for each product: the product name, the company name of the product supplier (use the suppliers table), the category name, the unit price, and the quantity per unit.\n',
    validator=val_practice_2,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'suppliers', 'categories']
)


## Time-constrained reports

Good job! Another very common type of report focuses on **a specific time period**.

In business reporting, questions like these come up all the time:
- How many orders were placed last month?
- What were the sales this year?
- How did performance change over time?

Let’s look at a simple example:

```sql
SELECT
  COUNT(*)
FROM orders
WHERE order_date >= '2016-07-01'
  AND order_date < '2016-08-01';
````

This query counts the number of orders placed in **July 2016**.

Here’s how it works:

* `WHERE` limits the rows to a specific date range
* `COUNT(*)` counts how many orders fall within that period

---

### Working with dates

In SQL, dates are written inside **single quotes** and are typically formatted as:

```
YYYY-MM-DD
```

This format may feel unfamiliar if you’re used to other date conventions, so it’s worth paying attention.

Also keep in mind:

* date handling can vary slightly between database engines
* formats and behaviors may depend on configuration

When in doubt, always check the database documentation.


In [12]:
# @title Practice 3
base_practice_3 = make_df_validator_nospoilers(
    expected_hash='53e54191a3f50721dcdffc38383108448ef4f145c9ad2652f898e138d79fc882',
    required_cols=['number_of_employees'],
    expected_rows=1,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_3 = base_practice_3

make_sql_runner(
    conn,
    runner_id="practice_3",
    description_md='### Practice 3\nCount the number of employees hired in 2013. Name the result number_of_employees.\n',
    validator=val_practice_3,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['employees']
)


## Computations for multiple objects

Well done! In real business reports, we rarely compute metrics for just one object.

Much more often, we want to calculate the **same metric for many objects at once**.

For example:
👉 How many items are in each order?

```sql
SELECT
  o.order_id,
  COUNT(*) AS order_items_count
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.order_id BETWEEN 10200 AND 10300
GROUP BY o.order_id;
````

Let’s walk through the logic:

1. join `orders` with `order_items` to connect orders with their items
2. filter to a specific range of orders using `WHERE`
3. group rows by `order_id`
4. use `COUNT(*)` to count how many items belong to each order

The result is a compact report where:

* each row represents one order
* the computed value (`order_items_count`) summarizes that order

This pattern — **join → group → aggregate** — is extremely common in reporting queries.


In [13]:
# @title Practice 4
base_practice_4 = make_df_validator_nospoilers(
    expected_hash='f49379c165cdbb636f0db8d1ef408c0aace86b22841f60c5ea5b839120af3897',
    required_cols=['supplier_id', 'company_name', 'products_count'],
    expected_rows=28,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_4 = base_practice_4

make_sql_runner(
    conn,
    runner_id="practice_4",
    description_md='### Practice 4\nShow each supplier_id alongside the company_name and the number of products they supply (as the products_count column). Use the products and suppliers tables.\n',
    validator=val_practice_4,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'suppliers']
)


## Total order value

Good job! In sales reporting, one of the most common questions is:

👉 *How much was a given order worth?*

Let’s see how we can calculate that in SQL.

```sql
SELECT
  SUM(unit_price * quantity) AS total_price
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.order_id = 10250;
````

This query calculates the **total value of order 10250** (before any discounts).

Here’s what’s happening:

* we join `orders` with `order_items` to access the individual items
* for each item, we multiply `unit_price` by `quantity`
* `SUM()` adds up those values to produce the total order price

Just like `COUNT()`, `SUM()` is a fundamental tool in business reporting — especially when working with sales, revenue, and costs.


In [14]:
# @title Practice 5
base_practice_5 = make_df_validator_nospoilers(
    expected_hash='d188a972763b9d709672813b880543493b4ca639b6c3b025d2c6a3c83d1406f0',
    required_cols=['total_price', 'total_price_after_discount'],
    expected_rows=1,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_5 = base_practice_5

make_sql_runner(
    conn,
    runner_id="practice_5",
    description_md='### Practice 5\nThe template code shows the query from the explanation. The Northwind store offers its customers discounts for some products. The discount for each item is stored in the discount column of the order_items table. (For example, a **0.20** discount means that the customer pays **1 - 0.2 = 0.8** of the original price.) Your task is to add a second column named total_price_after_discount.\n',
    validator=val_practice_5,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'suppliers'],
    default_sql='SELECT\n  SUM(unit_price * quantity) AS total_price\nFROM orders o\nJOIN order_items oi \n  ON o.order_id = oi.order_id\nWHERE o.order_id = 10250;\n'
)


## Order values for multiple orders

Good job!

We already know how to calculate the total value of **one** order.  
In real-world reports, though, we usually need to do this for **many orders at once**.

Here’s how that looks in SQL:

```sql
SELECT
  o.order_id,
  c.company_name AS customer_company_name,
  SUM(unit_price * quantity) AS total_price
FROM orders o
JOIN customers c
  ON o.customer_id = c.customer_id
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.ship_country = 'France'
GROUP BY o.order_id, c.company_name;
````

This query produces a report that shows:

* each order shipped to France
* the company that placed the order
* the total value of that order (before discounts)

Let’s focus on the key idea:

We’re using `GROUP BY` because we want **one row per order**, not one row per order item.

Even though grouping by `order_id` alone would uniquely identify each order, we also include `company_name` in the `GROUP BY`.

Why?

Because **every column in `SELECT` that is not aggregated** must appear in the `GROUP BY` clause.
Otherwise, SQL wouldn’t know which value to display for that column.

This rule is fundamental and you’ll encounter it often in reporting queries.


In [15]:
# @title Practice 6
base_practice_6 = make_df_validator_nospoilers(
    expected_hash='328992b7cea4c69cd3fb8a1eea6c78e002c5f44417aa2acc3922d8fa95288b7e',
    required_cols=['employee_id', 'first_name', 'last_name', 'orders_count'],
    expected_rows=10,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_6 = base_practice_6

make_sql_runner(
    conn,
    runner_id="practice_6",
    description_md='### Practice 6\nWe want to know the number of orders processed by each employee. Show the following columns: employee_id, first_name, last_name, and the number of orders processed as orders_count.\n',
    validator=val_practice_6,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['orders', 'employees']
)


In [16]:
# @title Practice 7
base_practice_7 = make_df_validator_nospoilers(
    expected_hash='89ec44d2e59b8bcd906ad166958fff85a1879538cbf7cf861879516b20979426',
    required_cols=['category_id', 'category_name', 'category_total_value'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_7 = base_practice_7

make_sql_runner(
    conn,
    runner_id="practice_7",
    description_md="### Practice 7\nHow much are the products in stock in each category worth? Show three columns: category_id, category_name, and category_total_value. You'll calculate the third column as the sum of unit prices multiplied by the number of units in stock for all products in the given category.\n",
    validator=val_practice_7,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'categories']
)


## Grouping by the correct columns

Perfect! Now that we know how to compute metrics for multiple business objects, let’s look at a very common pitfall.

Suppose we want to count how many orders each employee at Northwind has handled.

You might be tempted to write something like this:

```sql
SELECT
  e.first_name,
  e.last_name,
  COUNT(*) AS orders_count
FROM orders o
JOIN employees e
  ON o.employee_id = e.employee_id
GROUP BY e.first_name,
  e.last_name;
````

At first glance, this query looks fine.

But there’s a problem.

Employee names are **not guaranteed to be unique**.
If two employees share the same first and last name, their orders would be grouped together — producing incorrect results.

---

### Fixing the query

The solution is simple: group by a **unique identifier**.

```sql
SELECT
  e.employee_id,
  e.first_name,
  e.last_name,
  COUNT(*) AS orders_count
FROM orders o
JOIN employees e
  ON o.employee_id = e.employee_id
GROUP BY e.employee_id,
  e.first_name,
  e.last_name;
```

By adding `employee_id`:

* each employee is uniquely identified
* orders are counted correctly
* results remain stable even if names are duplicated

---

### Rule to remember

When grouping data:

> **Always group by columns that uniquely identify the business object.**

Following this principle helps you avoid subtle mistakes that are easy to miss in reports.


In [17]:
# @title Practice 8
base_practice_8 = make_df_validator_nospoilers(
    expected_hash='50e54562a4e10d41ada628498fe9e74dff10d190d722970ae765ba0ef3e01028',
    required_cols=['customer_id', 'company_name', 'orders_count'],
    expected_rows=53,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_8 = base_practice_8

make_sql_runner(
    conn,
    runner_id="practice_8",
    description_md='### Practice 8\nCount the number of orders placed by each customer. Show the customer_id, company_name, and orders_count columns.\n',
    validator=val_practice_8,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['products', 'categories']
)


## Choosing information to show

Excellent! Let’s revisit the correct query from the previous section:

```sql
SELECT
  e.employee_id,
  e.first_name,
  e.last_name,
  COUNT(*) AS orders_count
FROM orders o
JOIN employees e
  ON o.employee_id = e.employee_id
GROUP BY e.employee_id,
  e.first_name,
  e.last_name;
````

Here, we group by `employee_id` to ensure correctness.

However, that **doesn’t mean we must display it**.

The following query is still perfectly valid:

```sql
SELECT
  e.first_name,
  e.last_name,
  COUNT(*) AS orders_count
FROM orders o
JOIN employees e
  ON o.employee_id = e.employee_id
GROUP BY e.employee_id,
  e.first_name,
  e.last_name;
```

Even though `employee_id` is used for grouping, it doesn’t appear in the final output and that’s completely fine.


### Important rule

We already know that:

> Every column in `SELECT` that is **not aggregated** must appear in `GROUP BY`.

But the reverse is **not** true.

> You do **not** have to select every column that appears in `GROUP BY`.

This gives you flexibility to:

* group by technical identifiers (IDs)
* display only user-friendly fields (names, labels)

A very common and very useful reporting pattern.


In [18]:
# @title Practice 9
base_practice_9 = make_df_validator_nospoilers(
    expected_hash='02460f0410828021e83d198789b5756f207a4c11141b80b5b669840e9f64e110',
    required_cols=['company_name', 'total_paid'],
    expected_rows=20,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_9 = base_practice_9

make_sql_runner(
    conn,
    runner_id="practice_9",
    description_md='### Practice 9\nWhich customers paid the most for orders made in June 2016 or July 2016? Show two columns:\n\n* company_name\n* total_paid, calculated as the total price (after discount) paid for all orders made by a given customer in June 2016 or July 2016.\n\nSort the results by total_paid in descending order.\n',
    validator=val_practice_9,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['orders', 'order_items', 'customers']
)


## Review of the COUNT() function

Well done! When building business reports, it’s important to clearly understand the difference between  
`COUNT(*)` and `COUNT(column_name)`.

Let’s look at a practical example.

Suppose we want to know:
- how many orders exist per country
- how many of those orders have already been shipped

```sql
SELECT
  ship_country,
  COUNT(*) AS all_orders,
  COUNT(shipped_date) AS shipped_orders
FROM orders
GROUP BY ship_country;
````

Here’s the key difference:

* `COUNT(*)` counts **all rows** for each `ship_country`
* `COUNT(shipped_date)` counts only rows where `shipped_date` is **not NULL**

In this database, a `NULL` value in `shipped_date` means the order has **not been shipped yet**.

So `COUNT(shipped_date)` effectively counts **only shipped orders**.

---

### Key takeaway

* `COUNT(*)` → how many records exist?
* `COUNT(column)` → how many records have a value?

Understanding this distinction allows you to turn technical details (`NULL` values) into meaningful business metrics.


In [19]:
# @title Practice 10
base_practice_10 = make_df_validator_nospoilers(
    expected_hash='c2c07231a1284b4070bcbef62448a7b47d5bdf461a4f06574be70538d5961ad8',
    required_cols=['all_customers_count', 'customers_with_fax_count'],
    expected_rows=1,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_10 = base_practice_10

make_sql_runner(
    conn,
    runner_id="practice_10",
    description_md='### Practice 10\nCount the total number of customers and all those with a fax number. Show two columns: all_customers_count and customers_with_fax_count\n',
    validator=val_practice_10,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['customers']
)


## Counting distinct objects

Good job! Let’s look at another common reporting scenario.

Suppose we want to count how many items are in each order but with a twist.

The same product can appear **multiple times** in a single order.  
So we may want to know:
- how many product entries exist in the order
- how many **unique products** were ordered

Here’s a query that gives us both:

```sql
SELECT
  order_id,
  COUNT(product_id) AS products_count,
  COUNT(DISTINCT product_id) AS unique_products_count
FROM order_items
GROUP BY order_id;
````

Let’s break it down:

* `COUNT(product_id)` counts **all product rows** in the order
* `COUNT(DISTINCT product_id)` counts only **unique products**

If the same `product_id` appears more than once in an order,
`COUNT(DISTINCT product_id)` will be **lower** than `COUNT(product_id)`.

This situation can occur, for example, when a customer adds the same product to an order multiple times — even though a `quantity` column exists.

---

### Key takeaway

* `COUNT(column)` → how many entries?
* `COUNT(DISTINCT column)` → how many unique objects?

This distinction is essential when translating raw data into meaningful business metrics.


In [20]:
# @title Practice 11
base_practice_11 = make_df_validator_nospoilers(
    expected_hash='8b36ee836e2c7302d1d8bc6d52be3c6efeeea5310987ce0a2c3b9c549aa6ad3d',
    required_cols=['number_of_customers'],
    expected_rows=1,
    sort_rows=True,
    sort_cols=True,
    exact_cols=False,
    hide_missing_cols=True,
    hide_row_count=False,
)

val_practice_11 = base_practice_11

make_sql_runner(
    conn,
    runner_id="practice_11",
    description_md='### Practice 11\nShow the number of unique customers (as number_of_customers) that had orders shipped to Spain.\n',
    validator=val_practice_11,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['orders']
)


## Summary

It’s time to wrap things up for this part. Let’s quickly review the key takeaways:

- SQL reports often require **joining multiple tables** to bring related information together.
- Every column in `SELECT` that is **not** used with an aggregate function must appear in the `GROUP BY` clause.
- Not every column used in `GROUP BY` needs to be shown in the final `SELECT` output.
- Be careful when using `COUNT()` — especially the difference between:
  - `COUNT(*)`
  - `COUNT(column)`
  - `COUNT(DISTINCT column)`

These principles form the foundation of most real-world SQL reports.  
Mastering them will make complex reporting queries much easier to write — and to debug.


In [21]:
# @title In-class assignment 1
make_sql_runner(
    conn,
    runner_id="in_class_1",
    description_md='### In-class assignment 1\nHow many distinct products are there in all orders shipped to France? Name the result distinct_products.\n',
    validator=None,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['orders', 'order_items']
)


In [22]:
# @title In-class assignment 2
make_sql_runner(
    conn,
    runner_id="in_class_2",
    description_md='### In-class assignment 2\nShow three kinds of information about product suppliers:\n\n1. all_suppliers (the total number of suppliers)\n2. suppliers_region_assigned (the total number of suppliers who are assigned to a region)\n3. unique_supplier_regions (the number of unique regions suppliers are assigned to)\n',
    validator=None,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['suppliers']
)


In [23]:
# @title In-class assignment 3
make_sql_runner(
    conn,
    runner_id="in_class_3",
    description_md='### In-class assignment 3\nFor each employee, compute the total order value before discount from all orders processed by this employee between 5 July 2016 and 31 July 2016. Ignore employees without any orders processed. Show the following columns: first_name, last_name, and sum_orders. Sort the results by sum_orders in descending order.\n',
    validator=None,
    sol_sql=None,
    select_only=True,
    dedupe=True,
    schema_tables=['orders', 'order_items', 'employees']
)
